<h2>June 18-22, 2018 / Exoplanets.org Work</h2>
<i>Peter Forshay</i>

Import the necessary Python libraries to start.

In [1]:
from astropy.coordinates import SkyCoord
import astropy.units as u
import os
import pandas as pd
import sqlite3

<h3>Getting Data</h3>
Set up some global variables with important filenames.  CSV files are available to download from each of the three main sites I used (Exoplanets.org, the NASA Exoplanet Archive, and Exoplanet.eu).  I included only confirmed planets with each CSV request.  Save these files as you like and update the file paths here.

In [2]:
EOD_PATH = "exoplanets.csv"
EU_PATH = "exoplanet_eu_catalog.csv"
ARC_PATH = "nasa_archive_confirmed.csv"

Use Panda's built-in 'read_csv' function to read the CSV files into DataFrames.  The NASA Archive file begins with several rows of column definitions, all commented with '#'.

In [3]:
EOD_FRAME = pd.read_csv(EOD_PATH, dtype="str")
EU_FRAME = pd.read_csv(EU_PATH)
ARC_FRAME = pd.read_csv(ARC_PATH, comment='#')

It'll be a lot easier to examine our results in a SQLite file, as opposed to more CSV's.

In [4]:
def create_sqlite_file():
    """
    Create and connect to a new SQLite database file.
    """
    
    # Provide a filename and erase any pre-existing
    # stale file.
    filename = "exoplanet_tables.db"
    if os.path.isfile(filename):
        os.remove(filename)
    
    conn = sqlite3.connect(filename)
    
    return conn

Connect to our SQLite database and write our first table with the full EOD contents.

In [5]:
CONN = create_sqlite_file()
EOD_FRAME.to_sql("EOD", CONN)

<h3>Identify Exoplanets Already Entered in EOD</h3>
Since we're trying to compare the EOD contents with those of other archives, we'll want to aggregate all the names of exoplanets already present in the EOD.

In [6]:
def get_all_eod_names(eod_frame):
    """
    Collect target names from all possible name fields in
    the EOD contents.
    
    :param eod_frame:  The current EOD contents.
    :type eod_frame:  Pandas.DataFrame
    """
    
    name_fields = ["NAME",
                   "OTHERNAME",
                   "JSNAME",
                   "EANAME",
                   "CPSNAME",
                   "SMENAME",
                   "TABLENAME",
                   "SIMBADNAME",
                   "ETDNAME"]
    all_names = []
    
    for field in name_fields:
        
        # Some of these fields may be defunct.  If confirmed, list
        # can be trimmed and 'try' removed.
        try:
            all_names.extend(eod_frame[field].tolist())
        except KeyError:
            continue
            
    # Filter out any None entries and return the unique list.
    all_names = filter(None, all_names)
    return list(set(all_names))

In [7]:
EOD_NAMES = get_all_eod_names(EOD_FRAME)

We also have a list of exoplanets we wish to exclude in a text file.  We can add these exoplanets to our current list and pretend these are already present in EOD to exclude them from any missing entry searches.

In [8]:
def read_skip_planets_txt(path):
    """
    Read a text file containing exoplanet names to omit
    from EOD.
    
    :param path:  File path to the text file.
    :type path:  str
    """
    
    skip = []
    
    with open(path, 'r') as skipfile:
        for line in skipfile:
            skip.append(line.strip())
            
    return skip

In [9]:
SKIP_PATH = "skip_planets.txt"
EOD_NAMES.extend(read_skip_planets_txt(SKIP_PATH))

Between the three main catalogs, there are a number of different naming conventions, capitalizations, hyphenations, etc.  When looking for matches, we want to avoid mismatches by eliminating all whitespace, making everything lower-case, and cutting out anything that isn't a letter or number.

In [10]:
def simple_str(name):
    """
    Reduce a provided string to lower-case letters and numbers.
    
    :param name:  The target name to reduce.
    :type name:  str
    """
    
    name = str(name).strip().lower()
    name = "".join(n for n in name if n.isalnum())
    
    return name

In [11]:
EOD_NAMES = [simple_str(n) for n in EOD_NAMES]
EOD_NAMES = list(set(EOD_NAMES))

<h3>Search for Exoplanets Not Currently in EOD</h3>
With our list of simplified, unique exoplanet names present in EOD, we can now search for entries in the other two catalogs that are not in our list.

In [12]:
def new_from_catalog(catalog, name_field, eod_names):
    """ 
    Compare a catalog with a list of EOD target names.  Also provide 
    the column to use for names within the catalog.
    
    :param catalog:  The ingested data from an outside catalog to
                     be scanned for new exoplanets.
    :type catalog:  Pandas.DataFrame
    
    :param name_field:  The column name to find exoplanet names
                        in catalog.
    :type name_field:  str
    
    :param eod_names:  A list of simplified target names already
                       present in EOD.
    :type eod_names:  list
    """
    
    cat_names = catalog[name_field].map(lambda x: simple_str(x))
    missing = catalog[~(cat_names.isin(eod_names))]
    
    return missing

In [13]:
EU_NEW_PLANETS = new_from_catalog(EU_FRAME, "# name", EOD_NAMES)
ARC_NEW_PLANETS = new_from_catalog(ARC_FRAME, "pl_name", EOD_NAMES)

We'll also add these results tables to our SQLite database.

In [14]:
EU_NEW_PLANETS.to_sql("EU entries not found in EOD", CONN)
ARC_NEW_PLANETS.to_sql("Arc entries not found in EOD", CONN)

/Users/pforshay/anaconda/envs/jwst-dev2/lib/python3.6/site-packages/pandas/core/generic.py:1362: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  chunksize=chunksize, dtype=dtype)


Next, we'll want to find which of these missing entries are present in both the NASA Archive and Exoplanet.eu.  We can do this by comparing the simplified names found in each of our missing-entry DataFrames.

In [15]:
def merge_missing_frames(arc_missing, eu_missing):
    """
    Merge two DataFrames of exoplanets not found in EOD from
    different sources.
    
    :param arc_missing:  Exoplanets found in the NASA Exoplanet
                         Archive not found in EOD.
    :type arc_missing:  Pandas.DataFrame
    
    :param eu_missing:  Exoplanets found in Exoplanet.eu not
                        found in EOD.
    :type eu_missing:  Pandas.DataFrame
    """
    
    eu_name = "# name"
    nasa_name = "pl_name"
    
    # Make sure the name fields in both frames are simplified.
    for n in arc_missing.index:
        arc_missing.at[n, nasa_name] = simple_str(arc_missing.at[n, nasa_name])
    for m in eu_missing.index:
        eu_missing.at[m, eu_name] = simple_str(eu_missing.at[m, eu_name])
    
    # Merge the two frames based on the name fields.
    merged_frame = pd.merge(arc_missing,
                            eu_missing,
                            how='inner',
                            left_on=nasa_name,
                            right_on=eu_name)
    
    # We'll pick out some select columns to examine, rather
    # than keeping all columns from both frames
    merged_frame = merged_frame[["pl_name",
                                 "# name",
                                 "ra_x",
                                 "ra_y",
                                 "dec_x",
                                 "dec_y",
                                 "st_vj",
                                 "mag_v",
                                 "pl_bmassj",
                                 "mass_sini",
                                 "pl_radj",
                                 "radius",
                                 "pl_orbper",
                                 "orbital_period",
                                 "pl_orbsmax",
                                 "semi_major_axis",
                                 "pl_orbeccen",
                                 "eccentricity",
                                 "pl_def_reflink",
                                 "pl_disc_reflink",
                                 ]]
    
    return merged_frame

In [16]:
NEW_PLANETS = merge_missing_frames(ARC_NEW_PLANETS, EU_NEW_PLANETS)

It's also possible that some of these missing entries are present in EOD under mistaken target names.  We'll try to look for these by comparing target coordinates.  The downside of this step is you may eliminate new exoplanets around hosts already entered into EOD, so the following function is able to either exclude targets with small separations or create a new table with potentially conflicting close targets.

In [17]:
def interpret_coords(coord_tup, unit_ra, unit_dec):
    """
    Translate coordinates read from exoplanet catalogs into SkyCoord
    objects.
    
    :param coord_tup:  A pair of RA & Dec coordinates.
    :type coord_tup:  tuple
    
    :param unit_ra:  Units for the RA coordinate.
    :type unit_ra:  astropy.units
    
    :param unit_dec:  Units for the Dec coordinate.
    :type unit_dec:  astropy.units
    """
    
    ra, dec = coord_tup
    if str(ra) == "nan" or str(dec) == "nan":
        coords = None
    else:
        coords = SkyCoord(ra, dec, unit=(unit_ra, unit_dec))
    return coords


def check_ra_dec(eod, missing, drop_far=True):
    """
    Compare coordinates of targets found in EOD with those of targets
    from another catalog.
    
    :param eod:  The current EOD contents.
    :type eod:  Pandas.DataFrame
    
    :param missing:  A set of search results from another catalog.
    :type missing:  Pandas.DataFrame
    
    :param drop_far:  Specify if we are excluding targets with large
                      separations from EOD targets.
    :type drop_far:  bool
    """
    
    # Get all EOD RA & DEC pairs into a list of SkyCoord objects.
    eod_ra = eod["RA"].tolist()
    eod_dec = eod["DEC"].tolist()
    eod_coords = list(tuple(zip(eod_ra, eod_dec)))
    eod_coords = [interpret_coords(c, u.hourangle, u.deg) for c in eod_coords]
    
    # Turn this list of coordinates into a dictionary, using 
    # integer RA values (in deg.) as keys.
    eod_dict = {}
    for c in eod_coords:
        if c is None:
            continue
        eod_ra = int(c.ra.deg)
        if eod_ra in eod_dict.keys():
            eod_dict[eod_ra].append(c)
        else:
            eod_dict[eod_ra] = [c]
            
    # Add new columns to the DataFrame to hold new results.
    missing.insert(len(missing.columns), "checked_coords", None)
    missing.insert(len(missing.columns), "min_distance", "Too far")
    
    far_targets = []
    for n, row in missing.iterrows():
        # Use "ra" if we are dropping large separations from
        # an EOD DataFrame.  Use "ra_x" if we are dropping 
        # small separations from an merged missing entries
        # frame.
        ra = "ra" if drop_far else "ra_x"
        dec = "dec" if drop_far else "dec_x"
        coords = (row[ra], row[dec])
        coords = interpret_coords(coords, u.deg, u.deg)
        
        # Record the coordinates for double-checking.
        missing.at[n, "checked_coords"] = str(coords)
        
        # Get the integer RA in degrees value for the current
        # missing entry.  Then include +/- 1 deg. for searching.
        ra_deg = int(coords.ra.deg)
        ra_deg = (ra_deg-1, ra_deg, ra_deg+1)
        
        # Look for matching RA keys in the eod_dict and add
        # any results to the list we'll compute distances for.
        nearby = []
        for ra in ra_deg:
            if ra in eod_dict.keys():
                nearby.extend(eod_dict[ra])
                
        # If nothing is nearby, add this to our list of 
        # targets with large separations.
        if len(nearby) == 0:
            far_targets.append(n)
            continue
            
        # Compute distances from the current target with all
        # potential matches found and take the minimum.
        distance = [c.separation(coords).arcsecond for c in nearby]
        distance = min(distance)
        missing.at[n, "min_distance"] = distance
        
        # If the distance is >10" (arbitrary) add this
        # to our list of far targets.
        if distance > 10:
            far_targets.append(n)
            
    # If drop_far is True, we're making a new DataFrame of
    # entries that don't match any EOD names but may be the
    # same target.
    if drop_far:
        all_nearby = missing.drop(far_targets)
        final_frame = all_nearby[["# name",
                                  "alternate_names",
                                  "star_name",
                                  "star_alternate_names",
                                  "ra",
                                  "dec",
                                  "checked_coords",
                                  "min_distance",
                                  ]]
    # If drop_far is False, we just want to filter our current
    # missing DataFrame for targets with large EOD separations.
    else:
        final_frame = missing[missing.index.isin(far_targets)]
    return final_frame

In [18]:
NEW_PLANETS = check_ra_dec(EOD_FRAME, NEW_PLANETS, drop_far=False)

Write our final DataFrame of missing exoplanet entries to our SQLite file and close the connection.

In [19]:
NEW_PLANETS.to_sql("Missing from EU & Arc w/ no nearby EOD", CONN)
CONN.close()

/Users/pforshay/anaconda/envs/jwst-dev2/lib/python3.6/site-packages/pandas/core/generic.py:1362: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  chunksize=chunksize, dtype=dtype)


<h3>Results</h3>
We now have a DataFrame populated with exoplanets that appear with identical simplified names in both the Exoplanet.eu and NASA Exoplanet Archive catalogs, but are not found in the EOD, and are at least 10 arcseconds away from any target currently in the EOD.  In this DataFrame, we provide side-by-side comparisons of a number of parameters from the NASA Archive ('_x') and Exoplanet.eu ('_y').  Overall, this provides a set of candidates for inclusion in the EOD with a high level of confidence that they are not currently members of the EOD.  Matching parameters that are identical or very close between our two reference sources may indicate the strongest candidates for inclusion.

In [20]:
NEW_PLANETS[:10]

,pl_name,# name,ra_x,ra_y,dec_x,dec_y,st_vj,mag_v,pl_bmassj,mass_sini,...,pl_orbper,orbital_period,pl_orbsmax,semi_major_axis,pl_orbeccen,eccentricity,pl_def_reflink,pl_disc_reflink,checked_coords,min_distance
0,24boob,24boob,217.157547,217.162500,49.844852,49.845000,5.580,5.60,0.910,0.9100,...,30.3506,30.3506,0.1900,0.1900,0.042,0.042,"<a refstr=""TAKARADA ET AL. 2018"" href=http://a...","<a refstr=""TAKARADA ET AL. 2018"" href=http://a...","<SkyCoord (ICRS): (ra, dec) in deg\n ( 217....",100671
1,bd032562b,bd032562b,177.564804,177.564803,2.760142,2.760131,9.580,9.58,6.400,6.4000,...,481.9000,481.9000,1.3000,1.3000,0.200,0.200,"<a refstr=""VILLAVER ET AL. 2017"" href=http://a...","<a refstr=""VILLAVER ET AL. 2017"" href=http://a...","<SkyCoord (ICRS): (ra, dec) in deg\n ( 177....",4049.53
2,bd152375b,bd152375b,176.482174,176.483333,14.616381,14.616389,10.310,10.30,1.061,1.0600,...,153.2200,153.2200,0.5760,0.5760,0.001,0.001,"<a refstr=""NIEDZIELSKI ET AL. 2016"" href=http:...","<a refstr=""NIEDZIELSKI ET AL. 2016"" href=http:...","<SkyCoord (ICRS): (ra, dec) in deg\n ( 176....",1818.18
3,bd152940b,bd152940b,240.095566,240.095800,15.546930,15.546900,9.046,9.19,1.110,1.1100,...,137.4800,137.4800,0.5390,0.5390,0.260,0.260,"<a refstr=""NOWAK ET AL. 2013"" href=http://adsa...","<a refstr=""NOWAK ET AL. 2013"" href=http://adsa...","<SkyCoord (ICRS): (ra, dec) in deg\n ( 240....",40858.9
4,bd20594b,bd20594b,53.650967,53.650000,20.599232,20.599167,11.038,11.00,0.070,0.0513,...,41.6855,41.6855,NaN,0.2410,0.000,0.000,"<a refstr=""STASSUN ET AL. 2017"" href=http://ad...","<a refstr=""ESPINOZA ET AL. 2016"" href=http://a...","<SkyCoord (ICRS): (ra, dec) in deg\n ( 53.6...",108217
5,bd49828b,bd49828b,45.640529,45.641667,49.730011,49.730000,9.380,9.38,1.600,1.6000,...,2590.0000,2590.0000,4.2000,4.2000,0.350,0.350,"<a refstr=""NIEDZIELSKI ET AL. 2015"" href=http:...","<a refstr=""NIEDZIELSKI ET AL. 2015"" href=http:...","<SkyCoord (ICRS): (ra, dec) in deg\n ( 45.6...",253924
6,bd061339b,bd061339b,88.251183,88.250000,-5.994844,-5.994700,9.700,6.69,0.027,0.0270,...,3.8728,3.8728,0.0428,0.0428,0.000,0.000,"<a refstr=""LO CURTO ET AL. 2013"" href=http://a...","<a refstr=""LO CURTO ET AL. 2013"" href=http://a...","<SkyCoord (ICRS): (ra, dec) in deg\n ( 88.2...",46933
7,bd061339c,bd061339c,88.251183,88.250000,-5.994844,-5.994700,9.700,6.69,0.170,0.1700,...,125.9400,125.9400,0.4350,0.4350,0.310,0.000,"<a refstr=""LO CURTO ET AL. 2013"" href=http://a...","<a refstr=""LO CURTO ET AL. 2013"" href=http://a...","<SkyCoord (ICRS): (ra, dec) in deg\n ( 88.2...",46933
8,bd114672b,bd114672b,278.370148,278.370833,-11.636039,-11.636111,9.990,10.02,0.530,0.5300,...,1667.0000,1667.0000,2.2800,2.2800,0.050,0.050,"<a refstr=""MOUTOU ET AL. 2015"" href=http://ads...","<a refstr=""MOUTOU ET AL. 2015"" href=http://ads...","<SkyCoord (ICRS): (ra, dec) in deg\n ( 278....",18123.4
10,corot24b,corot24b,101.922547,101.925000,-3.719301,-3.719444,NaN,NaN,0.018,NaN,...,5.1134,5.1134,0.0560,0.0560,0.000,0.000,"<a refstr=""ALONSO ET AL. 2014"" href=http://ads...","<a refstr=""ALONSO ET AL. 2014"" href=http://ads...","<SkyCoord (ICRS): (ra, dec) in deg\n ( 101....",2292.69


In [21]:
NEW_PLANETS.to_csv("new_exoplanets_to_add.csv")

In [22]:
from collections import Counter

def pull_out_papers(new_exoplanets_frame):
    """
    Get a dictionary with each paper referenced in the final
    DataFrame and count the number of references.
    
    :param new_exoplanets_frame:  The new exoplanet data.
    :type new_exoplanets_frame:  Pandas.DataFrame
    """

    paper_list = new_exoplanets_frame["pl_def_reflink"].tolist()
    ads = "http://adsabs.harvard.edu/abs/"
    links_list = []
    
    # Some links go to queries, while others go straight to
    # the paper.
    for p in paper_list:
        query = p.split("bibcode=")
        if len(query) == 1:
            bibcode = p.split("abs/")[1]
        elif len(query) == 2:
            bibcode = query[1]
        bibcode = bibcode.split(" ")[0]
        link = "".join([ads, bibcode])
        links_list.append(link)
        
    links_dict = Counter(links_list)
    for paper, num in links_dict.items():
        print(paper + " : {}".format(num))

In [23]:
pull_out_papers(NEW_PLANETS)

http://adsabs.harvard.edu/abs/2018arXiv180404008T : 1
http://adsabs.harvard.edu/abs/2017A&A...606A..38V : 1
http://adsabs.harvard.edu/abs/2016A&A...588A..62N : 2
http://adsabs.harvard.edu/abs/2013ApJ...770...53N : 2
http://adsabs.harvard.edu/abs/2017AJ....153..136S : 5
http://adsabs.harvard.edu/abs/2015ApJ...803....1N : 1
http://adsabs.harvard.edu/abs/2013A%26A...551A..59L : 4
http://adsabs.harvard.edu/abs/2015A&A...576A..48M : 4
http://adsabs.harvard.edu/abs/2014A&A...567A.112A : 2
http://adsabs.harvard.edu/abs/2017arXiv170307416D : 4
http://adsabs.harvard.edu/abs/2018arXiv180503466C : 1
http://adsabs.harvard.edu/abs/2017AJ....153..191S : 1
http://adsabs.harvard.edu/abs/2017A&A...602A..88A : 10
http://adsabs.harvard.edu/abs/2017A&A...608A..63P : 1
http://adsabs.harvard.edu/abs/2016A&A...593A.117A : 2
http://adsabs.harvard.edu/abs/2017A&A...597A.108S : 1
http://adsabs.harvard.edu/abs/2017arXiv170506537S : 1
http://adsabs.harvard.edu/abs/2017arXiv170901527N : 3
http://adsabs.harvard.edu